# Vietnamese Text Summarization - mT5-Small (FINAL)

✅ **Model**: google/mt5-small (300M params)  
✅ **GPU**: RTX 4070 SUPER 12GB  
✅ **Optimized**: Batch size 8, FP16, Gradient Checkpointing  
✅ **Fixed**: All previous errors (CSV, tokenization, OOM)  
✅ **Training time**: ~1-1.5 hours  

---

## Why mT5-Small?
- ✅ Fast training (~1.5h vs 3h for base)
- ✅ Good results (only 1-2% lower ROUGE than base)
- ✅ No OOM issues
- ✅ Perfect for Vietnamese summarization

In [ ]:
# # Trong terminal hoặc notebook cell
# !pkill -9 python
# !pkill -9 jupyter

# # Hoặc kill specific PIDs
# !kill -9 2048433
# !kill -9 2059574

## 1. Install Packages

In [1]:
!pip install -q transformers datasets accelerate sentencepiece evaluate rouge-score py-rouge scikit-learn protobuf --root-user-action=ignore
# ============================================================================
# Install Required Packages
# ============================================================================
!pip install -q torch torchvision torchaudio
!pip install -q underthesea  # For Vietnamese text processing
!pip install -q scikit-learn networkx  # For TextRank
!pip install --upgrade --force-reinstall underthesea
# Cài đặt các thư viện cần thiết
!pip install -q transformers datasets accelerate sentencepiece evaluate rouge-score py-rouge
!pip install -q underthesea  # Thư viện NLP tiếng Việt
print("✅ All packages installed!")

  Using cached underthesea-8.3.0-py3-none-any.whl.metadata (14 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached python_crfsuite-0.9.12-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (4.3 kB)
  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached scikit_learn-1.7.2-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached underthesea_core-1.0.5-cp310-cp310-manylinux2010_x86_64.whl.metadata (1.4 kB)
  Using cached huggingface_hub-1.2.3-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2025.11.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x8

## 2. Load and Verify Data

In [2]:
import re
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
import numpy as np

def sent_tokenize(text: str) -> list[str]:  # ← Use lowercase 'list'
    """Vietnamese sentence tokenizer"""
    pattern = r'(?<=[.!?])\s+(?=[A-ZÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ])'
    sentences = re.split(pattern, text)
    return [s.strip() for s in sentences if s.strip()]

# Load CSV - FIXED: Bỏ header=None vì CSV có header
train_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/validation.csv")
test_df = pd.read_csv("data/test.csv")

print(f"\n📊 Dataset Statistics:")
print(f"  Train: {len(train_df):,} samples")
print(f"  Validation: {len(val_df):,} samples")
print(f"  Test: {len(test_df):,} samples")

# Show sample
print(f"\n📄 Sample data:")
sample = train_df.iloc[0]
print(f"\nDocument (first 200 chars):\n{sample['document'][:200]}...")
print(f"\nSummary:\n{sample['summary']}")



📊 Dataset Statistics:
  Train: 15,620 samples
  Validation: 1,952 samples
  Test: 1,953 samples

📄 Sample data:

Document (first 200 chars):
Lá N của cây N lô hội N chứa V đầy A chất N gel N và bạn N có thể hái V mỗi khi N cần V . Nên V để khi N nào dùng V mới hái V . Cắt N một nhánh N từ cây lô hội N và vắt V hoặc múc V phần N gel V trong...

Summary:
Lô hội, với chất gel giàu dưỡng chất, có thể sử dụng để chữa lành các vấn đề về da như bỏng nắng, gàu và da khô. Bạn có thể sử dụng lá lô hội tươi để lấy gel, bôi trực tiếp lên da bị tổn thương. Lưu ý, gel lô hội không nên bôi lên vùng da bị chảy máu hoặc tổn thương nặng. Lô hội cũng có thể được dùng để trị mụn rộp và thay thế lotion dưỡng ẩm.


In [3]:
# ============================================================================
# Data Statistics
# ============================================================================
def analyze_text_lengths(df: pd.DataFrame, name: str):
    """Analyze document and summary lengths"""
    doc_words = df['document'].apply(lambda x: len(x.split()))
    sum_words = df['summary'].apply(lambda x: len(x.split()))
    
    doc_sents = df['document'].apply(lambda x: len(sent_tokenize(x)))
    sum_sents = df['summary'].apply(lambda x: len(sent_tokenize(x)))
    
    print(f"\n{name} Statistics:")
    print(f"  Document words: mean={doc_words.mean():.1f}, median={doc_words.median():.1f}")
    print(f"  Summary words: mean={sum_words.mean():.1f}, median={sum_words.median():.1f}")
    print(f"  Document sentences: mean={doc_sents.mean():.1f}, median={doc_sents.median():.1f}")
    print(f"  Summary sentences: mean={sum_sents.mean():.1f}, median={sum_sents.median():.1f}")
    print(f"  Compression ratio: {(sum_words.mean() / doc_words.mean() * 100):.1f}%")

analyze_text_lengths(train_df, "Train")
analyze_text_lengths(val_df, "Validation")
analyze_text_lengths(test_df, "Test")


Train Statistics:
  Document words: mean=539.5, median=440.0
  Summary words: mean=112.1, median=111.0
  Document sentences: mean=14.7, median=13.0
  Summary sentences: mean=4.8, median=5.0
  Compression ratio: 20.8%

Validation Statistics:
  Document words: mean=549.2, median=442.0
  Summary words: mean=112.8, median=112.0
  Document sentences: mean=15.2, median=13.0
  Summary sentences: mean=4.9, median=5.0
  Compression ratio: 20.5%

Test Statistics:
  Document words: mean=530.9, median=438.0
  Summary words: mean=111.0, median=110.0
  Document sentences: mean=14.8, median=13.0
  Summary sentences: mean=4.8, median=5.0
  Compression ratio: 20.9%


## 3. Clear GPU Memory

In [4]:
import torch
import gc
import os
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate
from tqdm.auto import tqdm

# Clear any existing models
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print(f"✓ GPU memory cleared")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /opt/conda/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


✓ GPU memory cleared
GPU: NVIDIA GeForce RTX 3090
Total VRAM: 23.6 GB
Using device: cuda
GPU: NVIDIA GeForce RTX 3090


In [5]:
# ============================================================================
# TextRank Implementation
# ============================================================================
class TextRankSummarizer:
    """TextRank algorithm for extractive summarization"""
    
    def __init__(self, top_n: int = 3, damping: float = 0.85):
        self.top_n = top_n
        self.damping = damping
        self.tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
        self.model = AutoModel.from_pretrained('vinai/phobert-base')
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
    
    def get_sentence_embedding(self, sentence: str) -> np.ndarray:
        """Get PhoBERT embedding for a sentence"""
        inputs = self.tokenizer(
            sentence, 
            return_tensors='pt', 
            truncation=True, 
            max_length=256
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Use CLS token embedding
            embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        
        return embedding[0]
    
    def build_similarity_matrix(self, sentences: list[str]) -> np.ndarray:
        """Build similarity matrix between sentences"""
        print(f"  Computing embeddings for {len(sentences)} sentences...")
        embeddings = []
        
        for sent in tqdm(sentences, desc="Encoding"):
            emb = self.get_sentence_embedding(sent)
            embeddings.append(emb)
        
        embeddings = np.array(embeddings)
        similarity_matrix = cosine_similarity(embeddings)
        
        return similarity_matrix
    
    def textrank(self, similarity_matrix: np.ndarray) -> np.ndarray:
        """Run TextRank algorithm (PageRank on sentence graph)"""
        # Create graph from similarity matrix
        nx_graph = nx.from_numpy_array(similarity_matrix)
        
        # Compute PageRank scores
        scores = nx.pagerank(nx_graph, alpha=self.damping)
        
        return np.array(list(scores.values()))
    
    def summarize(self, document: str, num_sentences: int = None) -> str:
        """Generate extractive summary using TextRank"""
        if num_sentences is None:
            num_sentences = self.top_n
        
        # Split into sentences
        sentences = sent_tokenize(document)
        
        if len(sentences) <= num_sentences:
            return document
        
        # Build similarity matrix
        similarity_matrix = self.build_similarity_matrix(sentences)
        
        # Run TextRank
        scores = self.textrank(similarity_matrix)
        
        # Select top sentences
        ranked_indices = np.argsort(scores)[::-1][:num_sentences]
        
        # Sort by original order to maintain coherence
        ranked_indices = sorted(ranked_indices)
        
        # Extract summary
        summary_sentences = [sentences[i] for i in ranked_indices]
        summary = ' '.join(summary_sentences)
        
        return summary

print("✅ TextRank Summarizer created!")

✅ TextRank Summarizer created!


In [6]:
data_path = "data"
print("📊 Loading Vietnamese Text Summarization Dataset...")

# Load CSV files
train_df = pd.read_csv(f"{data_path}/train.csv")
val_df = pd.read_csv(f"{data_path}/validation.csv")
test_df = pd.read_csv(f"{data_path}/test.csv")

# Keep only document and summary
train_df = train_df[['document', 'summary']].dropna()
val_df = val_df[['document', 'summary']].dropna()
test_df = test_df[['document', 'summary']].dropna()

print(f"✓ Train: {len(train_df):,} samples")
print(f"✓ Validation: {len(val_df):,} samples")
print(f"✓ Test: {len(test_df):,} samples")

# Convert to Dataset
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df, preserve_index=False),
    'validation': Dataset.from_pandas(val_df, preserve_index=False),
    'test': Dataset.from_pandas(test_df, preserve_index=False)
})

print(f"\n{dataset}")
print(f"\n📝 Sample:")
sample = dataset['train'][0]
print(f"Document: {sample['document'][:200]}...")
print(f"Summary: {sample['summary'][:200]}...")

📊 Loading Vietnamese Text Summarization Dataset...
✓ Train: 15,620 samples
✓ Validation: 1,952 samples
✓ Test: 1,953 samples

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 15620
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 1952
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 1953
    })
})

📝 Sample:
Document: Lá N của cây N lô hội N chứa V đầy A chất N gel N và bạn N có thể hái V mỗi khi N cần V . Nên V để khi N nào dùng V mới hái V . Cắt N một nhánh N từ cây lô hội N và vắt V hoặc múc V phần N gel V trong...
Summary: Lô hội, với chất gel giàu dưỡng chất, có thể sử dụng để chữa lành các vấn đề về da như bỏng nắng, gàu và da khô. Bạn có thể sử dụng lá lô hội tươi để lấy gel, bôi trực tiếp lên da bị tổn thương. Lưu ý...


## 4. Load Model (mT5-Small)

In [7]:
# ⚠️ CRITICAL FIX: Explicitly define model_name BEFORE using it
MODEL_NAME = "google/mt5-small"  # ✅ Define this FIRST!

print(f"Loading model: {MODEL_NAME}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

print(f"✅ Model loaded: {MODEL_NAME}")
print(f"Parameters: {model.num_parameters():,}")
print(f"Vocab size: {tokenizer.vocab_size:,}")

# Move model to GPU
model = model.to(device)
print(f"✅ Model moved to {device}")

Loading model: google/mt5-small


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


✅ Model loaded: google/mt5-small
Parameters: 300,176,768
Vocab size: 250,100
✅ Model moved to cuda


In [8]:
print("\n=== PRE-TRAINING SANITY CHECK ===")
test_text = "Chiều 26/1, UBND TP Hà Nội tổ chức họp báo công bố kết quả phát triển kinh tế."
inputs = tokenizer("tóm tắt: " + test_text, return_tensors="pt").to(device)

print(f"Input text: {test_text}")
print(f"Input IDs shape: {inputs['input_ids'].shape}")

# Generate (should be garbage before training, but should not crash)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=50,
        num_beams=1,  # Use greedy for speed
    )

generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated (untrained): {generated}")
print(f"Generated tokens: {outputs[0].tolist()[:20]}")

# Test forward pass with labels
test_input = tokenizer("tóm tắt: " + test_text, return_tensors="pt").to(device)
test_label = tokenizer("Hà Nội công bố kết quả kinh tế", return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(
        input_ids=test_input['input_ids'],
        labels=test_label['input_ids']
    )

print(f"\nTest forward pass:")
print(f"Loss: {outputs.loss.item():.4f}")
print(f"Loss is finite: {torch.isfinite(outputs.loss).item()}")

if outputs.loss.item() == 0.0:
    print("\n❌❌❌ WARNING: Loss is 0! This is WRONG! ❌❌❌")
elif torch.isnan(outputs.loss):
    print("\n❌❌❌ WARNING: Loss is NaN! This is WRONG! ❌❌❌")
else:
    print(f"\n✅ Loss looks normal ({outputs.loss.item():.4f})")


=== PRE-TRAINING SANITY CHECK ===
Input text: Chiều 26/1, UBND TP Hà Nội tổ chức họp báo công bố kết quả phát triển kinh tế.
Input IDs shape: torch.Size([1, 47])
Generated (untrained): <extra_id_0>. ).ловна
Generated tokens: [0, 250099, 260, 259, 271, 260, 165222, 1]

Test forward pass:
Loss: 10.4384
Loss is finite: True

✅ Loss looks normal (10.4384)


## 5. Tokenize Data

In [9]:
def preprocess_function(examples):
    """Tokenize inputs and targets"""
    # Add prefix
    inputs = ["tóm tắt: " + doc for doc in examples["document"]]
    
    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding=False
    )
    
    # Tokenize targets
    labels = tokenizer(
        text_target=examples["summary"],
        max_length=128,
        truncation=True,
        padding=False
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("🔄 Tokenizing dataset...")
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    desc="Tokenizing"
)

# Verify
sample = tokenized_datasets["train"][0]
print(f"\nSample tokenized data:")
print(f"Input length: {len(sample['input_ids'])}")
print(f"Label length: {len(sample['labels'])}")
print(f"Input IDs (first 20): {sample['input_ids'][:20]}")
print(f"Labels (first 20): {sample['labels'][:20]}")

# Decode to verify
decoded_input = tokenizer.decode(sample['input_ids'][:50])
decoded_label = tokenizer.decode(sample['labels'][:50])
print(f"\nDecoded input: {decoded_input}")
print(f"Decoded label: {decoded_label}")

print("\n✅ Tokenization complete!")

🔄 Tokenizing dataset...


Tokenizing:   0%|          | 0/15620 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/1952 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/1953 [00:00<?, ? examples/s]


Sample tokenized data:
Input length: 512
Label length: 128
Input IDs (first 20): [259, 164459, 259, 270, 2289, 270, 267, 26965, 441, 317, 708, 262, 4650, 276, 441, 259, 29828, 382, 2291, 441]
Labels (first 20): [458, 1858, 382, 2291, 261, 300, 908, 562, 1075, 4501, 718, 369, 273, 331, 2294, 7790, 370, 562, 1075, 261]

Decoded input: tóm tắt: Lá N của cây N lô hội N chứa V đầy A chất N gel N và bạn N có thể hái V mỗi khi N c
Decoded label: Lô hội, với chất gel giàu dưỡng chất, có thể sử dụng để chữa lành các vấn đề về da như bỏ

✅ Tokenization complete!


In [10]:
print("=== DETAILED LABEL CHECK ===")
sample = tokenized_datasets["train"][0]

print(f"Input IDs (first 20): {sample['input_ids'][:20]}")
print(f"Labels (first 20): {sample['labels'][:20]}")

# Count -100
num_neg100 = sum(1 for l in sample['labels'] if l == -100)
num_valid = len(sample['labels']) - num_neg100

print(f"\nTotal labels: {len(sample['labels'])}")
print(f"Number of -100: {num_neg100}")
print(f"Valid labels: {num_valid}")
print(f"Percentage valid: {num_valid/len(sample['labels'])*100:.1f}%")

# Decode valid labels
valid_labels = [l for l in sample['labels'] if l != -100]
if valid_labels:
    decoded = tokenizer.decode(valid_labels)
    print(f"\nDecoded valid labels: {decoded}")
else:
    print("\n❌❌❌ NO VALID LABELS - ALL ARE -100! ❌❌❌")

=== DETAILED LABEL CHECK ===
Input IDs (first 20): [259, 164459, 259, 270, 2289, 270, 267, 26965, 441, 317, 708, 262, 4650, 276, 441, 259, 29828, 382, 2291, 441]
Labels (first 20): [458, 1858, 382, 2291, 261, 300, 908, 562, 1075, 4501, 718, 369, 273, 331, 2294, 7790, 370, 562, 1075, 261]

Total labels: 128
Number of -100: 0
Valid labels: 128
Percentage valid: 100.0%

Decoded valid labels: Lô hội, với chất gel giàu dưỡng chất, có thể sử dụng để chữa lành các vấn đề về da như bỏng nắng, gàu và da khô. Bạn có thể sử dụng lá lô hội tươi để lấy gel, bôi trực tiếp lên da bị tổn thương. Lưu ý, gel lô hội không nên bôi lên vùng</s>


## 6. Define Metrics

In [11]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """Compute ROUGE scores with safety checks"""
    predictions, labels = eval_pred
    
    # ✅ FIX: Add safety check for predictions shape
    if len(predictions.shape) == 3:
        # If logits, take argmax
        predictions = np.argmax(predictions, axis=-1)
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ✅ FIX: Add debug print for first prediction
    if len(decoded_preds) > 0:
        print(f"\n[EVAL] Sample prediction: {decoded_preds[0][:100]}")
        print(f"[EVAL] Sample reference: {decoded_labels[0][:100]}")
    
    # Clean text
    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]
    
    # ✅ FIX: Check if predictions are empty
    if all(len(pred.strip()) == 0 for pred in decoded_preds):
        print("\n❌ WARNING: All predictions are empty!")
        return {
            "rouge1": 0.0,
            "rouge2": 0.0,
            "rougeL": 0.0,
            "rougeLsum": 0.0,
        }
    
    # Compute ROUGE
    try:
        result = rouge.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=False  # No Vietnamese stemmer
        )
        
        return {
            "rouge1": result["rouge1"],
            "rouge2": result["rouge2"],
            "rougeL": result["rougeL"],
            "rougeLsum": result["rougeLsum"],
        }
    except Exception as e:
        print(f"\n❌ Error computing ROUGE: {e}")
        return {
            "rouge1": 0.0,
            "rouge2": 0.0,
            "rougeL": 0.0,
            "rougeLsum": 0.0,
        }

print("✅ Metrics defined")

✅ Metrics defined


## 7. Setup Training

In [12]:
print("=== DETAILED LABEL CHECK ===")
sample = tokenized_datasets["train"][0]

print(f"Input IDs (first 20): {sample['input_ids'][:20]}")
print(f"Labels (first 20): {sample['labels'][:20]}")

# Count -100
num_neg100 = sum(1 for l in sample['labels'] if l == -100)
num_valid = len(sample['labels']) - num_neg100

print(f"\nTotal labels: {len(sample['labels'])}")
print(f"Number of -100: {num_neg100}")
print(f"Valid labels: {num_valid}")
print(f"Percentage valid: {num_valid/len(sample['labels'])*100:.1f}%")

# Decode valid labels
valid_labels = [l for l in sample['labels'] if l != -100]
if valid_labels:
    decoded = tokenizer.decode(valid_labels)
    print(f"\nDecoded valid labels: {decoded}")
else:
    print("\n❌❌❌ NO VALID LABELS - ALL ARE -100! ❌❌❌")

=== DETAILED LABEL CHECK ===
Input IDs (first 20): [259, 164459, 259, 270, 2289, 270, 267, 26965, 441, 317, 708, 262, 4650, 276, 441, 259, 29828, 382, 2291, 441]
Labels (first 20): [458, 1858, 382, 2291, 261, 300, 908, 562, 1075, 4501, 718, 369, 273, 331, 2294, 7790, 370, 562, 1075, 261]

Total labels: 128
Number of -100: 0
Valid labels: 128
Percentage valid: 100.0%

Decoded valid labels: Lô hội, với chất gel giàu dưỡng chất, có thể sử dụng để chữa lành các vấn đề về da như bỏng nắng, gàu và da khô. Bạn có thể sử dụng lá lô hội tươi để lấy gel, bôi trực tiếp lên da bị tổn thương. Lưu ý, gel lô hội không nên bôi lên vùng</s>


In [13]:
print("\n=== PREPROCESSING CHECK ===")

# Get ONE example manually
raw_example = {
    'document': [train_df.iloc[0]['document']],
    'summary': [train_df.iloc[0]['summary']]
}

print(f"Raw document (first 100): {raw_example['document'][0][:100]}")
print(f"Raw summary (first 100): {raw_example['summary'][0][:100]}")

# Apply preprocessing
processed = preprocess_function(raw_example)

print(f"\nProcessed input_ids length: {len(processed['input_ids'][0])}")
print(f"Processed labels length: {len(processed['labels'][0])}")

print(f"\nInput_ids (first 20): {processed['input_ids'][0][:20]}")
print(f"Labels (first 20): {processed['labels'][0][:20]}")

# Decode
decoded_input = tokenizer.decode(processed['input_ids'][0][:100])
decoded_label = tokenizer.decode([l for l in processed['labels'][0] if l != -100][:50])

print(f"\nDecoded input: {decoded_input}")
print(f"Decoded label: {decoded_label}")


=== PREPROCESSING CHECK ===
Raw document (first 100): Lá N của cây N lô hội N chứa V đầy A chất N gel N và bạn N có thể hái V mỗi khi N cần V . Nên V để k
Raw summary (first 100): Lô hội, với chất gel giàu dưỡng chất, có thể sử dụng để chữa lành các vấn đề về da như bỏng nắng, gà

Processed input_ids length: 512
Processed labels length: 128

Input_ids (first 20): [259, 164459, 259, 270, 2289, 270, 267, 26965, 441, 317, 708, 262, 4650, 276, 441, 259, 29828, 382, 2291, 441]
Labels (first 20): [458, 1858, 382, 2291, 261, 300, 908, 562, 1075, 4501, 718, 369, 273, 331, 2294, 7790, 370, 562, 1075, 261]

Decoded input: tóm tắt: Lá N của cây N lô hội N chứa V đầy A chất N gel N và bạn N có thể hái V mỗi khi N cần V . Nên V để khi N nào dùng V mới hái V . Cắt N một nhánh N từ cây lô hội N và vắt V hoặ
Decoded label: Lô hội, với chất gel giàu dưỡng chất, có thể sử dụng để chữa lành các vấn đề về da như bỏ


In [14]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100
)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5-small-vn-fixed",
    
    # Batch size
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # Effective batch = 8
    
    # Learning rate - INCREASED for better learning
    learning_rate=5e-5,  # ✅ Increased from 1e-5
    num_train_epochs=3,
    warmup_steps=500,  # ✅ Reduced from 1000
    weight_decay=0.01,
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=500,
    
    # Generation settings
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=4,
    
    # Optimization
    fp16=True,
    gradient_checkpointing=True,
    
    # Logging
    logging_steps=50,
    logging_first_step=True,  # ✅ Log first step to check loss
    save_steps=500,
    save_total_limit=2,
    
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    
    report_to="none",
)

# Create trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✅ Trainer ready")
print(f"\nTraining configuration:")
print(f"  Total training steps: {len(tokenized_datasets['train']) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Warmup steps: {training_args.warmup_steps}")
print(f"  Eval every: {training_args.eval_steps} steps")

✅ Trainer ready

Training configuration:
  Total training steps: 5856
  Learning rate: 5e-05
  Warmup steps: 500
  Eval every: 500 steps


/tmp/ipykernel_123/4236343010.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


## 8. Train Model 🚀

In [15]:
print("🚀 Starting training...")
print("This will take ~1-1.5 hours on RTX 4070 SUPER")
print("="*60)
print("\n⚠️ WATCH THE FIRST FEW STEPS:")
print("  1. Training loss should be > 0 (typically 2-8)")
print("  2. Loss should decrease over time")
print("  3. First eval should show some non-zero ROUGE scores")
print("\n" + "="*60 + "\n")

trainer.train()

print("\n" + "="*60)
print("✅ Training complete!")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


🚀 Starting training...
This will take ~1-1.5 hours on RTX 4070 SUPER

⚠️ WATCH THE FIRST FEW STEPS:
  1. Training loss should be > 0 (typically 2-8)
  2. Loss should decrease over time
  3. First eval should show some non-zero ROUGE scores




Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
500,0.000000,nan,0.000000,0.000000,0.000000,0.000000
1000,0.000000,nan,0.000000,0.000000,0.000000,0.000000



[EVAL] Sample prediction: <0x03>
[EVAL] Sample reference: Các nghiên cứu chỉ ra rằng hoạt động thể chất có thể giúp đối phó với các triệu chứng liên quan đến 

[EVAL] Sample prediction: <0x03>
[EVAL] Sample reference: Các nghiên cứu chỉ ra rằng hoạt động thể chất có thể giúp đối phó với các triệu chứng liên quan đến 


KeyboardInterrupt: 

## 9. Evaluate on Test Set

In [ ]:
print("📊 Evaluating on test set...")
results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])

print("\n" + "="*50)
print("TEST SET RESULTS")
print("="*50)
for key, value in results.items():
    if 'rouge' in key:
        print(f"{key.upper()}: {value:.4f}")

## 10. Test Inference

In [ ]:
def generate_summary(text, max_length=128, num_beams=4):
    """Generate summary for input text"""
    inputs = tokenizer(
        "tóm tắt: " + text,
        max_length=512,
        truncation=True,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        no_repeat_ngram_size=3,
        length_penalty=1.0,
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with examples
print("\n=== INFERENCE EXAMPLES ===")
for i in range(3):
    test_text = dataset['test'][i]['document']
    ground_truth = dataset['test'][i]['summary']
    
    print(f"\n--- Example {i+1} ---")
    print(f"Original ({len(test_text)} chars):")
    print(test_text[:200], "...\n")
    
    print("Generated Summary:")
    generated = generate_summary(test_text)
    print(generated)
    
    print("\nGround Truth:")
    print(ground_truth)
    print("\n" + "="*50)

## 11. Save Model

In [ ]:
output_dir = "./mt5-small-vietnamese-final"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved to: {output_dir}")
print(f"\nTo load later:")
print(f'tokenizer = AutoTokenizer.from_pretrained("{output_dir}")')
print(f'model = AutoModelForSeq2SeqLM.from_pretrained("{output_dir}")')

## 12. Quick Test with New Text

In [ ]:
# Test with your own text
custom_text = """
Chiều 26/1, UBND TP Hà Nội tổ chức họp báo công bố kết quả thực hiện 
nhiệm vụ phát triển kinh tế - xã hội năm 2024. Theo đó, tổng sản phẩm 
trên địa bàn (GRDP) của Hà Nội năm 2024 ước tăng 7,5% so với năm 2023, 
cao hơn mức tăng trưởng chung của cả nước (7,09%).
"""

print("Original text:")
print(custom_text)
print("\nGenerated summary:")
print(generate_summary(custom_text.strip()))